In [3]:
from tempfile import mkdtemp
import shutil
dirpath = mkdtemp()
fileA = os.path.join(dirpath, 'testA.dat')
fileB = os.path.join(dirpath, 'testB.dat')
A = np.memmap(fileA, dtype='float64', mode='w+', shape=(47616, 55296, 3))
print(dirpath)
# A[:] = np.ones((47616, 55296, 3))
# del A
#shutil.rmtree(dirpath)

/tmp/tmp7bakompo


In [4]:
A[:] = np.ones((47616, 55296, 3))

In [ ]:
A

In [ ]:
A[:] = -np.log10(A[:])

In [9]:
arr = np.empty((47616*55296, 3), dtype='float64')
print(type(arr), type(A))

<class 'numpy.ndarray'> <class 'numpy.memmap'>


In [10]:
arr[:] = np.reshape(A, (-1, 3))
print(type(arr), type(A))

<class 'numpy.ndarray'> <class 'numpy.memmap'>


In [11]:
arr.shape

(2632974336, 3)

In [12]:
A.shape

(47616, 55296, 3)

In [15]:
del arr

In [8]:
A_ = np.memmap(fileA, dtype='float64', mode='r', shape=(47616, 55296, 3))
B = np.memmap(fileB, dtype='float64', mode='w+', shape=(47616*55296, 3))
B[:] = A_.reshape((-1, 3))

In [11]:
def get_item_from_disk():
    B_ = np.memmap(fileB, dtype='float64', mode='w+', shape=(47616*55296, 3))
    return B_

arr[:] = get_item_from_disk()

In [14]:
arr2 = np.empty((47616*55296, 3), dtype='float64')
arr2[:] = arr[:]

In [20]:
arr2[100, 1]
del arr, arr2

In [24]:
B_1 = np.memmap(fileA, dtype='float64', mode='r', shape=(47616*55296, 3))
B_1[100,2]

1.0

In [28]:
os.remove(fileA)

In [32]:
arr = np.empty((47616*55296, 3), dtype='float64')
arr[:] = B_1[:]

In [34]:
arr[20000, 2]

1.0

In [33]:
os.listdir(dirpath)

[]

In [ ]:
def color_deconv(rgb, channel, conv_matrix):
    """RGB to stain color space conversion using color deconvolution.
    Original implementation in scikit-image 
    https://github.com/scikit-image/scikit-image/blob/master/skimage/color/colorconv.py#L1350
    
    Parameters
    ----------
    rgb : array_like
        The image in RGB format, in a 3-D array of shape ``(.., .., 3)``.
    channel : integer of [0,2]
        The specific output channel to compute
    conv_matrix: ndarray
        The stain separation matrix as described by G. Landini [1]_.
    Returns
    -------
    out : ndarray
        The image in stain color channel, in a 2-D array of shape
        ``(.., ..)``.
    """
    rgb = img_as_float(rgb, force_copy=True)
    rgb += 2
    out_shape = rgb.shape[0:2]
    
    # Copy data in rgb into a memmap numpy array
    _rgb = np.memmap(fp, dtype='float64', mode='w+', shape=rgb.shape)
    _rgb[:] = rgb[:]
    del rgb
    
    # Compute log, reshape memmap array and load into a normal numpy array
    _rgb = (-np.log10(_rgb)).reshape((-1, 3))
    stains = np.empty(_rgb.shape, dtype='float64')
    stains[:] = _rgb[:]
    
    stains = stains @ (conv_matrix[:,channel].reshape(-1, 1))
    return np.reshape(stains, out_shape)